<a href="https://colab.research.google.com/github/akshatamadavi/data_mining/blob/main/clustering/04_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part D: DBSCAN Clustering using PyCaret

This notebook demonstrates DBSCAN (Density-Based Spatial Clustering of Applications with Noise) clustering using the PyCaret library. DBSCAN is a density-based clustering algorithm that can discover clusters of arbitrary shapes and identify outliers as noise points.

**Key Features of DBSCAN:**
- Does not require specifying the number of clusters beforehand
- Can identify outliers/noise in the data
- Works well with clusters of varying shapes and densities
- Requires two parameters: eps (neighborhood radius) and min_samples (minimum points to form a cluster)

In [ ]:
# Install PyCaret
!pip install pycaret --quiet

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, make_moons
from pycaret.clustering import *
import warnings
warnings.filterwarnings('ignore')

print('✓ All libraries imported successfully!')

## 2. Load and Prepare Dataset

We'll create a synthetic dataset with non-linear clusters (moons shape) to demonstrate DBSCAN's ability to identify complex cluster shapes.

## 1. Install and Import Required Libraries

First, we need to install PyCaret library and import necessary packages for data manipulation and visualization.